# Final Project - CS 181

**Deliverable #2**: Data Acquisition

**Name**: Hieu Tran

**Central Question**: Do countries with better environmental quality generally have stronger economies?

---
This is an interesting topic because it is often thought that richer and more developed countries tend to have higher standard of living and therefore higher environmental standard. This project seeks to explore whether there exists such a  relationship (between environmental quality and economic strength.) 

All data are collected from Wikipedia. Under its' Terms of Use, articles and media under free and open licenses can be "reused."

To answer the question, we will be using data for ecological footprint per country (Table 1) and nominal GDP per country (Table 2) and create, among other types of visualization, a scatterplot containing the 2 data points to identify any correlation between them.


---

In [1]:
# import required modules
import os
import os.path
import json
import io
import pandas as pd
from lxml import etree
import requests
from lxml import html
import util # module taken from class repo (util.py)
import sqlalchemy as sa

htmlparser =  etree.HTMLParser()

## Table 1: List of countries by ecological footprint
**Source**: https://en.wikipedia.org/wiki/List_of_countries_by_ecological_footprint

**Description**: This Wikipedia dataset contains data on the ecological footprint for each country provided by the Global Footprint Network's National Footprint Accounts. Numbers are given in global hectares per capita.

---
We will be using web scraping to extract this dataset and  convert it to a pandas DataFrame.

### **1.1.** Scrape the HTML and parse into XML Element

In [2]:
# Utilize web scraping method provided in class to read the HTML
resource_path = "/api/rest_v1/page/html/List_of_countries_by_ecological_footprint"

url = util.buildURL(resource_path, "en.wikipedia.org")
response = requests.get(url)
assert response.status_code == 200

# Parse the response content into an XML etree and get the root
tree1 = etree.parse(io.BytesIO(response.content), htmlparser)
root1 = tree1.getroot()

In [3]:
# Select the table of interest (containing Wikipedia's sortable class)
table1root = root1.xpath("//table[@class='wikitable sortable']")[0]

# Display the first 100 lines of the tree
util.print_xml(table1root, depth=30, nlines=100)

<table class='wikitable sortable' id='mwUw'>
  <tbody id='mwVA'>
    <tr id='mwVQ'>
      <th rowspan='2' id='mwVg'>Rank</th>
      <th rowspan='2' width='150pt' id='mwVw'>Country/regio
      <th id='mwWA'>Ecological
        <br id='mwWQ'></br>
      </th>
      <th id='mwWg'>Biocapacity</th>
      <th id='mwWw'>Biocapacity
        <br id='mwXA'></br>
        <br id='mwXQ'></br>
      </th>
      <th rowspan='2' id='mwXg'>Population
        <br id='mwXw'></br>
      </th>
      <th rowspan='2' width='100pt' id='mwYA'>Total biocapa
      <th rowspan='2' width='100pt' id='mwYQ'>Population (m
    </tr>
    <tr id='mwYg'>
      <th colspan='3' id='mwYw'>(gha/person)</th>
    </tr>
    <tr id='mwZA'>
      <td id='mwZQ'></td>
      <td id='mwZg'>World</td>
      <td id='mwZw'>2.75</td>
      <td id='mwaA'>1.63</td>
      <td id='mwaQ'>-1.12</td>
      <td id='mwag'>7753</td>
      <td id='mwaw'>-10400</td>
      <td id='mwbA'>2500</td>
    </tr>
    <tr id='mwbQ'>
      <td id='mwbg'>1</td>

In the next 2 sections, we will get the list of `header` and the `data` LoL to create a DataFrame.

### **1.2.** Acquire the column headers

Here, we will iterate through each header. For header that contains tag `<br>` (line break), we will append the text before and after the tag with a space in between.

In [4]:
headerroot_t1 = table1root.find(".//tr") # first row (headers)
headers_t1 = []

# Iterate through each header
for th in headerroot_t1.xpath(".//th"): 
    # Get the text (before <br>)
    text = th.text

    # Iterate over all <br> (line break) tags in the header and
    # append their tail attribute (everything after <br>) to the text
    for br in th.findall("br"):
        text += " " + (br.tail) # essentially replacing <br> with a space

    headers_t1.append(text.strip())
    
headers_t1

['Rank',
 'Country/region',
 'Ecological footprint',
 'Biocapacity',
 'Biocapacity deficit or reserve',
 'Population (millions)',
 'Total biocapacity deficit or reserve (gMha)',
 'Population (millions) for biocapacity to equal ecological footprint*']

### **1.3.** Acquire the data and create a DataFrame

Here, we will iterate over all data rows, extract the data of each row to a temporary list (`values`) and append it to the main list (`data_t1`)

In [5]:
rows_t1 = table1root.xpath(".//tr") # acquire all the rows
data_t1 = []

# Iterate over the remaining rows and extract the data
for row in rows_t1[2:]:
    values = []
    for td in row.xpath(".//td"):
        # Get the countries' names which, apart from "World",
        # are in children <a> of <td>
        if td.find(".//a") != None:
            values.append(td.find(".//a").text)
        else:
            values.append(td.text)
    data_t1.append(values) # append each row of values to the main list

# Create a DataFrame from the data and column headers
df1 = pd.DataFrame(data_t1, columns=headers_t1)
df1


,Rank,Country/region,Ecological footprint,Biocapacity,Biocapacity deficit or reserve,Population (millions),Total biocapacity deficit or reserve (gMha),Population (millions) for biocapacity to equal ecological footprint*
0,None,World,2.75,1.63,-1.12,7753,-10400,2500
1,1,Luxembourg,15.82,1.68,−14.14,0.63,−7.35,0.055221
2,2,Aruba,11.88,0.57,−11.31,0.17,−1.13,0.004798
3,3,Qatar,10.8,1.24,−9.56,2.05,−19.60,0.235373343
4,4,Australia,9.31,16.57,7.26,23.05,167.34,41.02454
...,...,...,...,...,...,...,...,...
185,184,Pakistan,0.79,0.35,−0.44,179.16,−78.83,79.37468
186,185,Bangladesh,0.72,0.38,−0.35,154.7,−54.15,81.64722
187,186,Haiti,0.61,0.27,−0.34,10.17,−3.46,4.501475
188,187,Timor-Leste,0.48,1.78,1.3,1.11,1.44,4.11625


For this project, we plan to use data from `Country/region` and `Ecological footprint` columns to get the ecological footprint of each country.

In [6]:
# Convert numbers currently in str to float
df1['Ecological footprint'] = df1['Ecological footprint'].astype(float)

# Select the columns
df1 = df1[["Country/region", "Ecological footprint"]]
df1 = df1.rename(columns={"Country/region": "countryname", "Ecological footprint": "ecologicalvalue"}) # Rename for later SQL processing

df1

,countryname,ecologicalvalue
0,World,2.75
1,Luxembourg,15.82
2,Aruba,11.88
3,Qatar,10.80
4,Australia,9.31
...,...,...
185,Pakistan,0.79
186,Bangladesh,0.72
187,Haiti,0.61
188,Timor-Leste,0.48


## Table 2: List of countries by GDP (nominal) per capita
**Source**: https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)_per_capita

**Description**: This Wikipedia dataset contains latest estimates on the nominal GDP per capita from 3 different sources: IMF, World Bank, and United Nations. All data are in USD.

---
We will be using the downloaded .html file (gdp.html), available in the `data` folder, to acquire the dataset and convert it to a pandas DataFrame.

### **2.1.** Parse the HTML into XML Element

In [7]:
# Parse the HTML into an XML etree and get the root
tree2 = etree.parse("./data/gdp.html", parser = htmlparser)
root2 = tree2.getroot()

In [8]:
# Select the table of interest (containing Wikipedia's sortable class)
table2root = root2.xpath("//*[contains(@class, 'wikitable sortable')]")[0]

# Display the first 100 lines of the tree
util.print_xml(table2root, depth=30, nlines=100)

<table class='wikitable sortable static-row-numbers plainro
  <caption>GDP (in USD) per capita by country (including
    <link rel='mw-deduplicated-inline-style' href='mw-data:
    <span ...>territories, and countries not included in th
  </caption>
  <tbody>
    <tr class='static-row-header' style='text-align:center;
      <th rowspan='2'>Country/Territory</th>
      <th rowspan='2'>
        <a href='/wiki/United_Nations_geoscheme' title='Uni
      </th>
      <th colspan='2'>
        <a href='/wiki/International_Monetary_Fund' title='
        <sup ...>
          <a href='#cite_note-GDP_IMF-4'>[4]</a>
        </sup>
        <sup ...>
          <a href='#cite_note-5'>[5]</a>
        </sup>
      </th>
      <th colspan='2'>
        <a href='/wiki/World_Bank' title='World Bank'>World
        <sup ...>
          <a href='#cite_note-worldbank-6'>[6]</a>
        </sup>
      </th>
      <th colspan='2'>
        <a href='/wiki/United_Nations' title='United Nation
        <sup ...>
         

We will be utilizing the same approach from the first database (get the list of `header` and the `data` LoL to create a DataFrame.) 

### **2.2.** Acquire the column headers

The second table contains two header rows (multi-level columns), with each sources of entry (IMF, World Bank, United Nations) spanning two columns for their respective `Estimate` and `Year` data. We will go through each line of headers and append each header to a list corresponding to the line it's in.

In [9]:
header1root_t2 = table2root.findall(".//tr")[0] # first line of headers
header1_t2 = []

for th in header1root_t2.xpath(".//th"): # iterate through each header
    # Get the headers' text 
    if th.find(".//a") != None: # apart from "Country/Territory", all headers' text are in children <a> of <th>
        text = th.find(".//a").text
        header1_t2.append(text.strip())
        # Duplicate the name of each source for the multi-level columns
        # ("UN Region" header is in <a> along with the sources and is excluded)
        if th.find(".//a").get("title")!="United Nations geoscheme": 
            header1_t2.append(text.strip())
    else: # i.e. "Country/Territory"
        text = th.text
        header1_t2.append(text.strip())


header2root_t2 = table2root.findall(".//tr")[1] # second line of headers
header2_t2 = []

for th in header2root_t2.xpath(".//th"): # iterate through each sub-header
    text = th.text
    header2_t2.append(text.strip())

# Insert blank text in place of the two columns that do not have sub-header
for i in [header1_t2.index("Country/Territory"), header1_t2.index("UN Region")]:
    header2_t2.insert(i, "")

headers_t2 = [header1_t2, header2_t2] # LoL containing both lines of headers
print(headers_t2)


[['Country/Territory', 'UN Region', 'IMF', 'IMF', 'World Bank', 'World Bank', 'United Nations', 'United Nations'], ['', '', 'Estimate', 'Year', 'Estimate', 'Year', 'Estimate', 'Year']]


### **2.3.** Acquire the data and create a DataFrame

Here, we will also be using the same approach as with Table 1 by iterating through each data row. Some special cases will be handled separately using different `for` loops and conditional statements.

In [10]:
# Get the data

rows_t2 = table2root.xpath(".//tr") # acquire all the rows

data_t2 = []

# Iterate over the remaining rows and extract the data
for row in rows_t2[3:]:
    values = []
    # Handle the "World" row, which has <th> as children instead of <td>
    for th in row.xpath(".//th"): 
        if th.get("style") == "text-align:left": # retrieve the <th> with text "World" 
            text = th.xpath("./text()")[-1] # (1)
            values.append(text)
            values.append("") # insert blank text representing the "UN Region" column
        else: # remaining data (i.e. 'Estimate' and 'Year')
            num = th.text.strip()
            if ',' in num:
                num = num.replace(',', '') # remove commas in values
                values.append(num)
            else:
                values.append(num)


    # Handle the remaining non-"World" rows (<td>)
    for td in row.xpath(".//td"):
        # Get the countries' names which are in children <a> of <td>
        if td.find(".//a") != None:
            # Deal with special cases of superscripts (Wikipedia's references) (2)
            if td.find(".//sup")!= None and td.find("./a") == None: 
                text = td.xpath("./text()")[-1] # (1)
                values.append(text.strip())
            else:
                values.append(td.find(".//a").text.strip())
        else:
            # Handle cases of non-available data
            if td.text == "—" or td.text == "—\n":
                # Insert None to represent empty datapoints ("Estimate" and "Year")
                values.extend([None for i in range(2)]) 
            else: # remaining data (i.e. 'Estimate' and 'Year')
                num = td.text.strip()
                if ',' in num:
                    num = num.replace(',', '') # # remove commas in values
                    values.append(num)
                else:
                    values.append(num)

            
    data_t2.append(values) # append each row of values to the main list

# Create a DataFrame from the data and column headers
df2 = pd.DataFrame(data_t2, columns=headers_t2)
df2


Country/Territory UN Region      IMF       World Bank        \
                                          Estimate  Year   Estimate  Year   
0                      Monaco *    Europe     None  None     234316  2021   
1               Liechtenstein *    Europe     None  None     157755  2020   
2                  Luxembourg *    Europe   127673  2022     133590  2021   
3                     Bermuda *  Americas     None  None     114090  2021   
4                     Ireland *    Europe   102217  2022     100172  2021   
..                          ...       ...      ...   ...        ...   ...   
218  Central African Republic *    Africa      496  2022        461  2021   
219              Sierra Leone *    Africa      494  2022        480  2021   
220               Afghanistan *      Asia      611  2020        369  2021   
221               South Sudan *    Africa      328  2022       1072  2015   
222                   Burundi *    Africa      293  2022        222  2021   

    United Nations        
          Estimate  Year  
0           234317  2021  
1           169260  2021  
2           133745  2021  
3           112653  2021  
4           101109  2021  
..             ...   ...  
218            461  2021  
219            505  2021  
220            373  2021  
221            400  2021  
222            311  2021  

[223 rows x 8 columns]

For this project, we plan to use data from `Country/Territory`, `UN Region` and `IMF` columns to get the GDP number of each country. Countries that does not have GDP data from the IMF will be excluded.

In [11]:
df2 = df2[["Country/Territory", "UN Region", "IMF"]]
df2['Country/Territory'] = df2['Country/Territory'].str.replace('\s\*', '', regex=True) # remove '*' in values

df2 = df2[[('Country/Territory', ''), ('UN Region', ''), ('IMF', 'Estimate')]] # disregard the 'Year' column
df2.columns = df2.columns.droplevel(1) # drop the multi-level columns
df2 = df2.rename(columns={"Country/Territory": "countryname", "UN Region": "region", "IMF": "gdpvalue"}) # rename for later SQL processing

# Convert numbers currently in str to int
df2['gdpvalue'] = df2['gdpvalue'].astype('Int64')

df2 = df2.dropna() # drop rows without data
df2 = df2.reset_index(drop=True)

df2

/var/folders/yz/b3jptcgn479804hrzzwpcf9r0000gn/T/ipykernel_44916/1166792592.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Country/Territory'] = df2['Country/Territory'].str.replace('\s\*', '', regex=True) # remove '*' in values


,countryname,region,gdpvalue
0,Luxembourg,Europe,127673
1,Ireland,Europe,102217
2,Norway,Europe,92646
3,Switzerland,Europe,92434
4,Qatar,Asia,82887
...,...,...,...
192,Central African Republic,Africa,496
193,Sierra Leone,Africa,494
194,Afghanistan,Asia,611
195,South Sudan,Africa,328


### Notes for parts of the code above:

**(1)**: Due to the way some of the data are configured, `.text` can't be used to grab the data. For example:


In [12]:
ex1 = """
<th colspan="2" style="text-align:left"> 
    <span class="flagicon" style="padding-left:25px;">&#160;</span>
    World
</th>
"""

Here, if we use `th.text` to try and grab "World", it will return `None`. This is due to how the `.text` function works, and does not return the value we are interested in when there is a children node before the text. In this case, to extract the text, we have to select the text node (at index -1) using `.xpath`. This is essentially the same scenario as with Table 1's headers.

**(2)**: In the case of Wikipedia's superscripts designed for citations, almost all of it are placed in `Year`. For example:

In [13]:
ex2a = """ 
<td>
    <sup id="cite_ref-9" class="reference">
        <a href="#cite_note-9">&#91;8&#93;</a>
    </sup>
    2021
</td>
"""

However, there's a one for the European Union that is in `Country/Territory`:

In [14]:
ex2b = """ 
<td style="text-align:left">
    <span class="flagicon" style="display:inline-block;width:25px;text-align:left">
        <img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/b7/Flag_of_Europe.svg/23px-Flag_of_Europe.svg.png" decoding="async" width="23" height="15" class="thumbborder" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/b/b7/Flag_of_Europe.svg/35px-Flag_of_Europe.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/b/b7/Flag_of_Europe.svg/45px-Flag_of_Europe.svg.png 2x" data-file-width="810" data-file-height="540" />
    </span>&#160;
    <a href="/wiki/Economy_of_the_European_Union" title="Economy of the European Union">
        European Union *
    </a>
    <sup id="cite_ref-EU_8-0" class="reference"> 
        <a href="#cite_note-EU-8">&#91;n 1&#93;
        </a>
    </sup>
</td>
"""

In this case, due to our implementation, we have to exclude this particular `<td>` from being included using its' `<a>` node which is only present for data in the `Country/Territory` column.

## SQL Processing


We will have a single table in our database comprised of the two datasets, named `indicators`. `countryname` will be the primary key (singleton), representing the names of countries as the independent variable, with the fields `region` (global region), `gdpvalue` (GDP per capita) and `ecologicalvalue` (ecological footprint) as the dependent variables.

In [15]:
df = df2.merge(df1, on= 'countryname')
df = df[df.countryname != 'World'] # exclude worldwide average data (not a country)
df = df.reset_index(drop=True)
df

,countryname,region,gdpvalue,ecologicalvalue
0,Luxembourg,Europe,127673,15.82
1,Ireland,Europe,102217,5.57
2,Norway,Europe,92646,4.98
3,Switzerland,Europe,92434,5.79
4,Qatar,Asia,82887,10.80
...,...,...,...,...
164,Madagascar,Africa,522,0.99
165,Central African Republic,Africa,496,1.24
166,Sierra Leone,Africa,494,1.24
167,Afghanistan,Asia,611,0.79


Our database is named `gdpeco.db`, located in the same directory as this notebook. We will create our table in the database and populate it using the combined dataset (`df`) above.

**Setting credentials**

In [16]:
def getsqlite_creds(dirname=".",filename="creds.json",source="sqlite"):
    """ Using directory and filename parameters, open a credentials file
        and obtain the two parts needed for a connection string to
        a local provider using the "sqlite" dictionary within
        an outer dictionary.  
        
        Return a scheme and a dbfile
    """
    assert os.path.isfile(os.path.join(dirname, filename))
    with open(os.path.join(dirname, filename)) as f:
        D = json.load(f)
    sqlite = D[source]
    return sqlite["scheme"], sqlite["dbdir"], sqlite["database"]

def buildConnectionString(source="sqlite_gdpeco"):
    scheme, dbdir, database = getsqlite_creds(source=source)
    template = '{}:///{}/{}.db'
    return template.format(scheme, dbdir, database)

In [17]:
# Build the connection string
cstring = buildConnectionString("sqlite_gdpeco")
print("Connection string:", cstring)

# Connect to the database
engine = sa.create_engine(cstring)
connection = engine.connect()

Connection string: sqlite:///./gdpeco.db


In [18]:
# SQL query to create our table
table_query = """
CREATE TABLE IF NOT EXISTS indicators (
    countryname VARCHAR(64) NOT NULL PRIMARY KEY,
    region VARCHAR(64),
    gdpvalue INT,
    ecologicalvalue FLOAT,
    CONSTRAINT unique_countryname UNIQUE (countryname)
)
"""

This database adheres to principles of sound DB design for the following reasons:
- The table `indicators` represents related entities;
- The primary key `countryname` is unique, not null, and cannot be changed;
- No values as fields (value-named attributes);
- No field name in a plural form or as a mashup (multi-part fields);
- No redundant info (each row is unique);
- No derived fields.

In [19]:
# Execute the query
try:
    connection.execute(table_query)
except sa.exc.SQLAlchemyError as err:
    print("CREATE of indicators failed:", str(err))

/var/folders/yz/b3jptcgn479804hrzzwpcf9r0000gn/T/ipykernel_44916/632199568.py:3: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  connection.execute(table_query)


In [20]:
# Insert our data into the database
df.to_sql("indicators", con=connection, if_exists="append", index=False)

169

In [21]:
# Close the connection
try:
    connection.close()
except:
    pass
del engine

Our process of storing the data into the database was relatively smooth with only one table required.

---